In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import csv
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import spectral_clustering
from sklearn.neighbors import kneighbors_graph

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
        
GROUND_TRUTH = '/kaggle/input/bsr-images/data-2/data/groundTruth/test'
RAW_DATA = '/kaggle/input/bsr-images/data-2/data/images/test'
OUT_CLUSTERING = '/kaggle/working'

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Dataset

In [ ]:
from PIL import Image
import scipy.io as sio
from matplotlib import cm, pyplot as plt

In [ ]:
means=[3,5,7,9,11] 

selected_images = [] # the IDs of the images to process
for _,_,filenames in os.walk(RAW_DATA):
    for filename in filenames:
        if '.jpg' in filename:
            selected_images.append(int(filename[:-4]))
selected_images = sorted(selected_images)[:50]
print(selected_images)

In [ ]:
# 100007, 100039 (321, 481, 3)
original_dim = [] # original dimensions of each image

def load_image(s, disp=True):
    ground_truths = [] # each element is an image, where each pixel has a label to identify its partition
    data_to_segment = [] # each element is an image represented as a flattened numpy array of pixels (i.e. 154401 pixels)
    i_subplot = 0
    
    d = os.path.join(OUT_CLUSTERING, str(s))
    os.makedirs(d, exist_ok=True)
    
    # image itself
    print('Reading image {}'.format(s))
    im = Image.open(os.path.join(RAW_DATA, '{}.jpg'.format(s)))
    mat = np.asarray(im)
    #print(mat.shape)
    #assert mat.shape == (321, 481, 3) Some images are 481 * 321!!!!!!!!!
    assert mat.shape[0] * mat.shape[1] == 154401
    original_dim.append((mat.shape[0], mat.shape[1]))
    
    #print(mat)
    data_to_segment = mat.reshape(154401, 3)
    
    # ground truth
    truths = sio.loadmat(os.path.join(GROUND_TRUTH, '{}.mat'.format(s)))['groundTruth'].squeeze()
    if disp:
        # set up figure 
        fig, axs = plt.subplots(1,1 + truths.shape[0], num=1, clear=True)
        fig.set_size_inches(25,15)
        axs[i_subplot].axis('off')
        axs[i_subplot].imshow(im)
    im.close()
    i_subplot += 1
    
    temp = []
    for truth in truths:
        uncovered = truth[0][0][0] # Are they afraid it'll run away? We need the segmentations not the boundaries anyway
        #assert uncovered.shape == (321, 481)
        assert uncovered.shape[0] * uncovered.shape[1] == 154401
        #print(uncovered)
        temp.append(np.array(uncovered).ravel())
        uncovered = uncovered /np.max(uncovered)
        #display(Image.fromarray(cm.gist_earth(uncovered, bytes=True)))
        if disp:
            axs[i_subplot].axis('off')
            #print(uncovered)
            axs[i_subplot].imshow(uncovered, cmap='gist_earth')
        i_subplot += 1
        
    if disp:
        fig.savefig(os.path.join(d, 'original_{}.png'.format(s)))
        
    #print(temp)
    ground_truths = temp
    return data_to_segment, ground_truths

In [ ]:

def display_output(idx, clusterings, method_name):
    image_name = selected_images[idx]
    d = os.path.join(OUT_CLUSTERING, str(image_name))
    
    mat_dict = {}
    
    fig, axs = plt.subplots(1,len(clusterings), clear=True)
    fig.set_size_inches(25,15)
    
    if len(clusterings) == 1:
        for (i, mat) in enumerate(clusterings):
            temp = np.array(mat).reshape(original_dim[idx])
            mat_dict[str(means[i])] = np.copy(temp)
            temp = temp/np.max(temp)
            #print(temp)
            axs.axis('off')
            axs.imshow(temp, cmap='gist_earth')
            #display(Image.fromarray(cm.gist_earth(temp, bytes=True)))
    else:
        for (i, mat) in enumerate(clusterings):
            temp = np.array(mat).reshape(original_dim[idx])
            mat_dict[str(means[i])] = np.copy(temp)
            temp = temp/np.max(temp)
            #print(temp)
            axs[i].axis('off')
            axs[i].imshow(temp, cmap='gist_earth')
            #display(Image.fromarray(cm.gist_earth(temp, bytes=True)))

    sio.savemat(os.path.join(d, str(s) + '.mat'), mat_dict)
    fig.savefig(os.path.join(d,"{}_{}.png".format(image_name, method_name)))

# Kmeans

In [ ]:
def diff(x, y):
    return abs(x-y)/max(x,y)

def k_means(image, k, threshold, max_iterations):

    np.random.seed(1)
    centroids = np.random.randn(k, image.shape[1])
    prev_loss = 1e9
    for i in range(max_iterations):
        dists_to_centroids = euclidean_distances(X=image, Y=centroids)
        assignment = np.argmin(dists_to_centroids, axis=1)
        loss = np.sum(np.linalg.norm(centroids[assignment] - image, axis=1)**2)
        if diff(loss, prev_loss) <= threshold:
            break
        prev_loss = loss

        for centroid_index in range(k):
            cent = image[np.where(assignment==centroid_index)]
            if cent.shape[0] > 0: 
                centroids[centroid_index] = np.mean(cent, axis=0)
        
    return assignment, centroids


In [ ]:
# cluster, c = k_means(data_to_segment[1],9,0.00001,10000)
# print(cluster)
# cl = KMeans(n_clusters=9, random_state=0).fit_predict(data_to_segment[1])
# print(cl)

## Clustering Evaluation

#### Contingency table calculation

In [ ]:
def calculate_contingency_matrix(clusters, classes, no_of_clusters, no_of_classes, clusters_labels):
    contingency_matrix = np.zeros((no_of_clusters, no_of_classes+1)).astype(int)
    data_length = len(clusters)
    for pixel in range(data_length):
        contingency_matrix[clusters_labels.index(clusters[pixel]), classes[pixel]-1] += 1
    for cluster in range(no_of_clusters):
        contingency_matrix[cluster, no_of_classes] = sum(contingency_matrix[cluster, :])
    return contingency_matrix

#### Conditional entropy calculation

In [ ]:
def conditional_entropy(clusters, classes, clusters_labels):
    no_of_clusters = len(np.unique(clusters))
    no_of_classes = len(np.unique(classes))
    no_of_pixels = len(clusters)
    contingency_matrix = calculate_contingency_matrix(clusters, classes, no_of_clusters, no_of_classes, clusters_labels)
    conditional_entropy_result = 0
    for cluster in contingency_matrix:
        conditional_entropy_given_cluster = 0
        for a_class in cluster[:no_of_classes]:
            fraction = a_class / cluster[-1]
            if fraction > 0:
                conditional_entropy_given_cluster -= fraction * math.log(fraction, 2)
        conditional_entropy_result += (cluster[-1] / no_of_pixels) * conditional_entropy_given_cluster
    return conditional_entropy_result

#### F-Measure calculation

In [ ]:
def f_measure(clusters, classes, clusters_labels):
    no_of_clusters = len(np.unique(clusters))
    no_of_classes = len(np.unique(classes))
    no_of_pixels = len(clusters)
    contingency_matrix = calculate_contingency_matrix(clusters, classes, no_of_clusters, no_of_classes, clusters_labels)
    f_measure_result = 0
    for cluster in contingency_matrix:
        max_index = list(cluster[:no_of_classes]).index(max(cluster[:no_of_classes]))
        purity = cluster[max_index] / cluster[-1]
        rec = cluster[max_index] / sum(contingency_matrix[:, max_index])
        f_measure_result += (2 * purity * rec) / (purity + rec)
    return f_measure_result / no_of_clusters

In [ ]:
def calculate_average_measures(cluster, ground_truths, clusters_labels, print_trace = False):
    average_conditional_entropy = 0
    average_f_measure = 0
    for gt_i in range(len(ground_truths)):
        ce = conditional_entropy(cluster, ground_truths[gt_i], clusters_labels)
        fm = f_measure(cluster, ground_truths[gt_i], clusters_labels)
        average_conditional_entropy += ce
        average_f_measure += fm
        if print_trace:
            print('Ground truth #{}: Conditional Entropy = {}, F-Measure = {}'.format(gt_i + 1, ce, fm))
    average_conditional_entropy /= len(ground_truths)
    average_f_measure /= len(ground_truths)
    return average_conditional_entropy, average_f_measure

# Main Code

In [ ]:
# For each k there is average CE and F_Measure on the whole dataset
dataset_average_conditional_entropy = [ 0 for k in means]
dataset_average_f_measure = [ 0 for k in means]
f_averages = [[] for k in means]
ce_averages = [[] for k in means]

for (j, s) in enumerate(selected_images):
    print("For image ",j+1)
    partitions=[]
    data_to_segment, ground_truths = load_image(s, disp=True)
    for i in range(len(means)):
        cluster, c = k_means(data_to_segment,means[i],0.00001,1000)
        print("###################")
        print("For k = ",means[i])
        partitions.append(cluster)
        average_conditional_entropy, average_f_measure = calculate_average_measures(cluster, ground_truths, list(np.unique(cluster)), True)
        f_averages[i].append((average_f_measure, s)) # lowest f measures = worst
        ce_averages[i].append((-average_conditional_entropy, s)) # highest ce = worst
        print('Average Conditional Entropy = {}, Average F-Measure = {}'.format(average_conditional_entropy, average_f_measure))
        # Division by 50 as selected images number = 50
        dataset_average_conditional_entropy[i] += average_conditional_entropy / 50
        dataset_average_f_measure[i] += average_f_measure / 50
        #print(cluster)
        
    display_output(j, partitions, 'kmeans')
    # Measures
    print(dataset_average_conditional_entropy)
    print(dataset_average_f_measure)
    

for (i, k) in enumerate(means):
    f_averages[i].sort()
    ce_averages[i].sort()
    print("K = {}".format(k))
    print("Worst by F-measure: " + ''.join('#{} '.format(s) for _,s in f_averages[i][:3]))
    print("Worst by CE: "+ ''.join('#{} '.format(s) for _,s in ce_averages[i][:3]))
    
    print("Best by F-measure: "+''.join('#{} '.format(s) for _,s in f_averages[i][-3:]))
    print("Best by CE: "+''.join('#{} '.format(s) for _,s in ce_averages[i][-3:]))

# Big Picture

In [ ]:
five_images_sample = [2018, 3063, 5096, 6046, 8068]

## a. K-means at K = 5 Vs Ground truth

In [ ]:
for (j, s) in enumerate(five_images_sample):
    print("For image ",j+1)
    data_to_segment, ground_truths = load_image(s, disp=True)
    cluster, c = k_means(data_to_segment, 5, 0.00001, 1000)
    average_conditional_entropy, average_f_measure = calculate_average_measures(cluster, ground_truths, list(np.unique(cluster)))
    print('Average Conditional Entropy = {}, Average F-Measure = {}'.format(average_conditional_entropy, average_f_measure))
    display_output(j, [cluster], 'kmeans')

## b. Normalized-cut at K=5 Vs Ground truth

In [ ]:
def save_clustering_results(results, name):
    np.savetxt(name + ".csv", results, delimiter =", ", fmt ='% s')

In [ ]:
def apply_normalized_cut(data_to_segment, file_name):
    affinity = kneighbors_graph(data_to_segment, 5, mode='connectivity')
    save_clustering_results(spectral_clustering(affinity, n_clusters=5), file_name)

In [ ]:
def read_clustering_from_file(file_name):
    with open(file_name, newline='\n') as file:
        reader = csv.reader(file)
        data = [int(row[0]) for row in reader]
    return data

In [ ]:
for (j, s) in enumerate(five_images_sample):
    print("For image ",j+1)
    data_to_segment, ground_truths = load_image(s, disp=True)
    clustering_result_path = '../input/nc-clustering-results/{}_nc_clustering.csv'.format(s)
    if not os.path.exists(clustering_result_path):
        apply_normalized_cut(data_to_segment, '{}_nc_clustering'.format(s))
        clustering_result_path = './{}_nc_clustering.csv'.format(s)
    cluster = read_clustering_from_file(clustering_result_path)
    average_conditional_entropy, average_f_measure = calculate_average_measures(cluster, ground_truths, list(np.unique(cluster)))
    print('Average Conditional Entropy = {}, Average F-Measure = {}'.format(average_conditional_entropy, average_f_measure))
    display_output(j, [cluster],'normalized_cut')

## c. Normalized-cut at K=5 Vs K-means at K = 5

In [ ]:
for (j, s) in enumerate(five_images_sample):
    print("For image ",j+1)
    data_to_segment, ground_truths = load_image(s, disp=False)
    clustering_result_path = '../input/nc-clustering-results/{}_nc_clustering.csv'.format(s)
    if not os.path.exists(clustering_result_path):
        apply_normalized_cut(data_to_segment, '{}_nc_clustering'.format(s))
        clustering_result_path = './{}_nc_clustering.csv'.format(s)
    nc_cluster = read_clustering_from_file(clustering_result_path)
    k_means_cluster, c = k_means(data_to_segment, 5, 0.00001, 1000)
    display_output(j, [nc_cluster], 'normalized_cut')
    display_output(j, [k_means_cluster], 'kmeans')